In [41]:
import requests
import time
import re
import pandas as pd
import numpy as np
import random
import pickle
import glob
import os
from bs4 import BeautifulSoup

In [42]:
#with open('kicktraq_data.pickle', 'rb') as f:
#    projects = pickle.load(f)

In [43]:
#projects=pd.read_csv('kicktraq_data.csv', names=['kickstarter_page','funding', 'campaign_dates', 'backers', 'category'])

In [44]:
#projects_from_pickle.campaign_dates.iloc[2]

##### Build projects dataframe from kicktraq page CSVs

In [45]:
page_numbers = range(2000,4000)
projects=pd.DataFrame(columns=['kickstarter_page','funding', 'campaign_dates', 'backers', 'category'])
col_types = {'kickstarter_page': 'str','funding': 'str', 'campaign_dates': 'str', 'backers': 'str', 'category': 'str'}

#for file in ['kicktraq_files/kicktraq_page_num21','kicktraq_files/kicktraq_page_num22']:
#for file in glob('kicktraq_files/*'):
for page_num in page_numbers:
    #test if file for this page already exists
    filename = 'kicktraq_files/kicktraq_page_num' + str(page_num)

    page_df = pd.read_csv(filename, dtype=col_types)
    projects = pd.concat([projects, page_df])

##### Check for duplicate rows in projects dataframe

In [46]:
#check for duplicates
print(projects.kickstarter_page.shape)
print(projects.kickstarter_page.unique().shape)
shape1 = projects.kickstarter_page.shape
shape2 = projects.kickstarter_page.unique().shape
if shape1 != shape2:
    projects.drop_duplicates(subset='kickstarter_page', inplace=True)

(29873,)
(29873,)


##### Initialize columns for kickstarter data with base values

In [47]:
projects['description_length'] = 0
projects['n_pledges'] = 0
projects['pledge_level_min'] = 0
projects['pledge_level_max'] = 0
projects['pledge_level_stddev'] = 0
projects['currency'] = '$'

In [48]:
def get_kickstarter_data(proj_series):
    url = 'https://www.kickstarter.com/' + proj_series.kickstarter_page
    #print(url)    
    
    #define filename for where to store html txt file
    page_id_regex = re.compile('projects\/.*\/(.*)\/')
    match = re.search(page_id_regex, proj_series.kickstarter_page)
    page_id = match.group(1)
    filename = r"kickstarter_files/{}.txt".format(page_id)
    
    #check if html txt file exists already
    if os.path.isfile(filename):
        with open( filename, "r") as f:
            page_text = f.read()
        soup = BeautifulSoup(page_text,'lxml')
        
        

    else:
        response = requests.get(url)
        print(url)

        #dealing with 404s 
        for i in range (1,10):
            if response.status_code == 200:
                break
            else:
                time.sleep(5)
                response = requests.get(url)
        page = response.text
        soup = BeautifulSoup(page,'lxml')

        #save soup as txt file
        with open( filename, "w") as f:
            f.write(str(soup.html))
            
    pledges = soup.find_all(class_='pledge__info')

    n_pledges = len(pledges)

    pledge_amounts=[]
    for pledge in pledges:
        pledge_amount = str(pledge.find(class_='money').text.replace('$',''))
        currency = ''
        #while the first character in pledge_amount is not an integer
        while not (re.match('\d',pledge_amount[0])):
            currency = currency + (pledge_amount[0])
            pledge_amount = pledge_amount[1:]
        if currency == '':
            currency = '$'
        proj_series['currency'] = currency
        pledge_amounts.append(int(pledge_amount.replace(',','')))

    try:
        descript_paragraphs = soup.find(class_="full-description js-full-description responsive-media formatted-lists").find_all('p')
        description_words = 0
        for par in descript_paragraphs:
            description_words += len(str(par.text).split())

        proj_series.description_length = description_words
        proj_series.n_pledges = n_pledges
        proj_series.pledge_level_min = min(pledge_amounts)
        proj_series.pledge_level_max = max(pledge_amounts)
        proj_series.pledge_level_stddev = np.std(pledge_amounts)
    except:
        print('something went wrong with' + filename)
        pass

    time.sleep(2)
        
    return proj_series


In [49]:
projects = projects.apply(get_kickstarter_data, axis=1)

https://www.kickstarter.com//projects/1756713787/drake-rug/
https://www.kickstarter.com//projects/1555317636/lyricspro-presentation-software/
https://www.kickstarter.com//projects/443183237/coleslaw-1/
https://www.kickstarter.com//projects/1792837597/quest-for-the-perfect-wing/
https://www.kickstarter.com//projects/191195726/turn-it-up-shared-bluetooth-music-app/
https://www.kickstarter.com//projects/1207881821/tip-toe-through-the-flowers/
https://www.kickstarter.com//projects/1648891931/the-angry-banana/
https://www.kickstarter.com//projects/431470588/pizza-time/
https://www.kickstarter.com//projects/433480266/the-rum-is-always-gone/
https://www.kickstarter.com//projects/858941951/fresh-food-through-family-farms/
https://www.kickstarter.com//projects/609688641/growing-krafty-max-originals/
https://www.kickstarter.com//projects/1853001654/space-journey/
https://www.kickstarter.com//projects/1693346270/project-dermatillomania-the-ebook-transformation/
https://www.kickstarter.com//projec

KeyboardInterrupt: 

In [ ]:
#col_types = {'kickstarter_page': 'str','funding': 'str', 'campaign_dates': 'str', 'backers': 'str', 'category': 'str', 
#             'description_length': 'int', 'n_pledges': 'int', 'pledge_level_min': 'int', 'pledge_level_max': 'int', 
#             'pledge_level_stddev': 'int', 'currency': 'str'}
#projects.astype(col_types, copy=False, errors='raise', encoding='utf-8')

In [ ]:
projects.shape

In [ ]:
with open('all_data_dirty_2000_3999.pickle', 'wb') as f:
    pickle.dump(projects, f)

In [ ]:
#projects = pd.to_csv('all_data_dirty.csv', index=False)

In [ ]:
#selenium

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

import os

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.binary_location = '/usr/bin/google-chrome'

chromedriver = "/bin/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver


driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver")


for page_num in page_numbers:
    url = 'https://www.kicktraq.com/archive/?page='+str(page_num)
    driver.get(url)
    time.sleep(10)